In [ ]:
#62103390
#遠藤愛期

#工夫点
#オブジェクト指向の利用
#GUIの作成（マスの大きさ統一）
#打てないマスをクリックできないようにした
#勝敗判定をしっかりつけた（全てのマスが埋まったときor自分の打つ手が無くなった時に判断される）
#ゲーム木探索アルゴリズム(NegaMax法)の利用
#以下のように強さを判定
# BOARD_SCORE = np.array( [[9,2,3,3,2,9],
#                          [2,1,1,1,1,2],
#                          [3,1,1,1,1,3],
#                          [3,1,1,1,1,3],
#                          [2,1,1,1,1,2],
#                          [9,2,3,3,2,9]])
#4つ角の横のマスは比較的弱い手とされているため1段階下げた
#授業中に配布されているファイルを最大限利用することで授業内容に対する理解を深めた
#このファイル以外にも、同じフォルダにあるファイルにも変更点を加えた

In [ ]:
import tkinter as tk

import numpy as np
import re

from Game     import TwoPlayersGame
from Player   import Human_Player, AI_Player
from Negamax  import Negamax
from Reversi import Reversi as AI

class Reversi_game():
    def __init__( self, cell ):
        self.board = None
        self.cell = cell
        self.board_myself = None
        self.turn = 0
        self.seq = 1
        self.result = None
        self.possible_cell_list = None
        self.judge = None
        self.message = None
        self.ui = None
        self.init_attributes()
    
    def init_attributes( self ):
        self.board = [ i  for i in range(self.cell)]
        self.board[14] = self.board[21] = "○"
        self.board[15] = self.board[20] = "×"
        self.board_myself = []
        self.turn = 0
        self.seq = 1
        self.result = []
        self.possible_cell_list = self.get_valid_moves()
        self.judge = "未定"
        self.message = ""
    
    def set_ui( self, ui ):
        self.ui = ui
    
    def changed( self ):
        if self.ui != None:
            self.ui.update()
    
    def changed_cell( self, position ):
        if self.ui != None:
            self.ui.update_cell( position )
    
    def is_int( self, data ):
        return( isinstance( data, int ) )
    
    def is_blank( self, position ):
        return( position >= 1 and position <= self.cell * self.cell \
                              and self.is_int( self.board[ position - 1 ] ) )
    
    def check_possibilities( self ):
        self.possible_cell_list = list( filter( self.is_int, self.board  ) )
        return( self.possible_cell_list )
    
    def put_stone( self, position ):
        if self.is_blank( position ):
            self.board[ position - 1 ] = self.get_stone_by_turn()
            return( True )
        else:
            return( False )
    
    def get_stone_in_cell( self, position ):
        stone = self.board[ position - 1 ]
        return( "　" if self.is_int( stone ) else stone )

    def start_game( self ):
        self.changed()
    
    def clear_game( self ):
        self.init_attributes()
        self.changed()
    
    def exit_game( self ):
        import sys
        sys.exit()

    def convert_to_label( self, number ):
        letters = ['A', 'B', 'C', 'D', 'E', 'F']
        col = number // 6
        row = number - col * 6 + 1
        return f"{letters[col]}{row}"
    
    def find_missing_numbers( self, arr ):
        missing_numbers = []
        for i in range( len( arr ) ):
            if not isinstance( arr[i], int ):
                missing_numbers.append( i )
        return missing_numbers
    
    def get_unique_values( self, array1, array2 ):
        unique_values = [x for x in array1 if x not in array2]
        return unique_values
    
    def put_stone_with_seq_control( self, position ):
        if self.result != []:
            self.message = "エラー: 既に勝敗がついているので，石を置けません"
            
        elif self.put_stone( position ):
            self.message = ""
            self.flip_opponent_stones( position )
            self.result = self.is_game_over()
            
            if self.result != []:
                if self.result[0] == self.result[1]:
                    self.judge = "!!引き分け!!"
                else:
                    self.judge = "全てのマスが埋まりました → " \
                                    + f"{'○' if self.result[0] > self.result[1] else '×'}の勝ち"
                    
            elif self.possible_cell_list == []:
                self.judge = "!!引き分け!!"
                
            else:
                self.judge = "未定"
                
                if self.turn == 0:
                    self.board_myself.append( self.convert_to_label( position - 1 ) )
                self.toggle_turn()
                self.possible_cell_list = self.get_valid_moves()
                self.seq += 1

                if self.turn ==1:
                    ai_algo = Negamax( 6 )
                    now_list = self.board_myself
                    self.game = AI( [ Human_Player(), AI_Player( ai_algo ) ] ).play( move_list=now_list )
                    array1 = self.game[2*len(now_list)-1][0].board
                    array2 = self.game[2*len(now_list)].board
                    changed_to_2_indices = np.argwhere((array2 - array1) == 2)
                    changed_to_2_indices = [index[0] * 6 + index[1] for index in changed_to_2_indices]
                    self.ui.model.put_stone_with_seq_control( changed_to_2_indices[0] + 1 )
                    
        else:
            pass
        self.changed()
    
    def flip_opponent_stones(self, cell):
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (-1, 1), (1, -1)]
        opponent_stone = "×" if self.board[cell - 1] == "○" else "○"

        for direction in directions:
            x, y = (cell - 1) % 6, (cell - 1) // 6
            x += direction[0]
            y += direction[1]
            flipped_stones = []

            while 0 <= x < 6 and 0 <= y < 6 and self.board[y * 6 + x] == opponent_stone:
                flipped_stones.append(y * 6 + x)
                x += direction[0]
                y += direction[1]

            if 0 <= x < 6 and 0 <= y < 6 and self.board[y * 6 + x] == self.board[cell - 1]:
                for stone in flipped_stones:
                    self.board[stone] = self.board[cell - 1]

    
    def is_game_over( self ):
        count = lambda symbol: sum(1 for cell in self.board if cell == symbol)
        
        if count("○") + count("×") == 36:
            self.result = [count("○"), count("×")]
            self.judge = "全てのマスが埋まりました → " \
                                    + f"{'○' if self.result[0] > self.result[1] else '×'}の勝ち"
            return [count("○"), count("×")]
        elif self.possible_cell_list == []:
            return [count("○"), count("×")]
        else:
            return []

    def get_valid_moves(self):
        valid_moves = []
        
        for i in range(len(self.board)):
            if self.put_stone( i + 1 ):
                now_list = self.board.copy()
                self.flip_opponent_stones( i + 1 )
                
                if now_list != self.board:
                    valid_moves.append(i)
                now_list[i] = i
                self.board[i] = i
                self.board = now_list
                
            else:
                valid_moves.append(i)       
                
        return valid_moves
    
    def get_stone_by_turn( self ):
        return( "○" if self.turn == 0 else "×" )
        
    def toggle_turn( self ):
        self.turn = ( self.turn + 1 ) % 2
    
    def pass_turn( self ):
        self.message = "パスをします．"
        self.toggle_turn()
        self.seq += 1
        self.changed()
    
class Reversi_gui():
    def __init__( self, model, master ):
        self.model = model
        self.master = master
        self.panel_view = None
        self.panel_control = None
        self.board_view = None
        self.label_seq = None
        self.label_turn = None
        self.label_possible_cell_list = None
        self.label_result = None
        self.label_judge = None
        self.label_message = None
        self.init_components()
        self.model.set_ui( self )
    
    def init_components( self ):
        self.panel_view    = self.init_panel_view( self.master )
        self.panel_control = self.init_panel_control( self.master )
    
    def init_panel_view_status( self, master ):
        self.panel_view_status = tk.Frame( master )
        self.panel_view_status.pack( side=tk.TOP )
        self.label_seq     = tk.Label( self.panel_view_status, text="t" )
        self.label_turn    = tk.Label( self.panel_view_status, text="" )
        self.label_possible_cell_list \
                           = tk.Label( self.panel_view_status, text="" )
        self.label_result = tk.Label( self.panel_view_status, text="" )
        self.label_judge   = tk.Label( self.panel_view_status, text="" )
        self.label_message = tk.Label( self.panel_view_status, text="" )
        self.label_seq.grid( row=0, column=0 )
        self.label_turn.grid( row=0, column=1 )
        self.label_possible_cell_list.grid( row=4, column=0, columnspan=2 )
        self.label_result.grid( row=5, column=0, columnspan=2 )
        self.label_judge.grid( row=6, column=0, columnspan=2 )
        self.label_message.grid( row=7, column=0, columnspan=2 )
        return( self.panel_view_status )
    
    def init_panel_view_board( self, master ):
        self.panel_view_board = tk.Frame( master )
        self.panel_view_board.pack( side=tk.TOP )
        self.board_view = []
        
        for i in range( 36 ):
            button_cell_text = "　"
            button_cell = \
                tk.Button( self.panel_view_board,
                        text=button_cell_text,
                        width=2, height=2,
                        command=self.create_callback_func_cell_clicked( i + 1 ) )
            button_cell.grid( row=int(i/6), column=i%6 )
            self.board_view.append( button_cell )
        return( self.panel_view_board )
    
    def init_panel_view( self, master ):
        self.panel_view = tk.Frame( master )
        self.panel_view.pack( side=tk.TOP )
        self.init_panel_view_status( self.panel_view )
        self.init_panel_view_board(  self.panel_view )
        return( self.panel_view )
    
    def init_panel_control( self, master ):
        self.panel_control = tk.Frame( master )
        self.panel_control.pack( side=tk.TOP )
        self.button_pass  = tk.Button( self.panel_control, text="パス",
                                       command=self.on_pass_clicked )
        self.button_clear = tk.Button( self.panel_control, text="クリア",
                                       command=self.on_clear_clicked )
        self.button_exit  = tk.Button( self.panel_control, text="終了",
                                       command=self.on_exit )
        self.button_pass.grid(  row=0, column=0 )
        self.button_clear.grid( row=0, column=1 )
        self.button_exit.grid(  row=0, column=2 )
        return( self.panel_control )
    
    def update( self ):
        self.update_status()
        self.update_message()
        self.update_board()
        
    def update_status( self ):
        self.label_seq[     "text" ] = f"手数 = {self.model.seq}"
        self.label_turn[    "text" ] = f"手番 = {self.model.get_stone_by_turn()}"
        self.label_possible_cell_list[ "text" ] = \
                        f"  着手可能なセル = {self.model.get_unique_values( self.model.possible_cell_list, self.model.find_missing_numbers(self.model.board) )}"
        count_circle = np.count_nonzero(np.array( self.model.board ) == "○" )
        count_cross = np.count_nonzero(np.array( self.model.board ) == "×" )
        self.label_result[   "text" ] = f"石の数 =　○ : × = {count_circle} : {count_cross}"
        self.label_judge[   "text" ] = f"勝敗判定 = {self.model.judge}"
    
    def update_message( self ):
        self.label_message[ "text" ] = f"{self.model.message}"
    
    def update_board( self ):
        for i, button in enumerate( self.board_view ):
            if i in self.model.possible_cell_list:
                button.config(state=tk.NORMAL)
            else:
                button.config(state=tk.DISABLED)
        for i in range( 36 ):
            self.board_view[ i ][ "text" ] = self.model.get_stone_in_cell( i + 1 )
    
    def update_cell( self, position ):
        self.board_view[ position -1 ][ "text" ] = self.model.board[ position - 1 ]
    
    def create_callback_func_cell_clicked( self, position ):
        def on_cell_clicked():
            self.model.put_stone_with_seq_control( position )
        return( on_cell_clicked )
    
    def on_pass_clicked( self ):
        self.model.pass_turn()
    
    def on_clear_clicked( self ):
        self.model.clear_game()
    
    def on_exit( self ):
        self.master.destroy()
        self.model.exit_game()

class App():
    def __init__( self ):
        self.main_window = tk.Tk()
        self.init_main_window()
        self.game = Reversi_game( cell=36 )
        self.game_gui = Reversi_gui( self.game, master=self.main_window )
        self.game.start_game()

    def init_main_window( self ):
        self.main_window.title( "オセロ（Reversi）" )

    def run( self ):
        self.main_window.mainloop()

    def destroy( self ):
        self.main_window.destroy()

def main():
    app = App()
    app.run()

if __name__ == "__main__":
    main()